In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

def preprocess_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

ds_train = ds_train.map(preprocess_image)
ds_test = ds_test.map(preprocess_image)

X_train, Y_train = [], []
for image, label in ds_train:
    X_train.append(image.numpy())
    Y_train.append(label.numpy())

X_test, Y_test = [], []
for image, label in ds_test:
    X_test.append(image.numpy())
    Y_test.append(label.numpy())

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# Convert the labels to one-hot encoded categorical format
num_classes = ds_info.features['label'].num_classes
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)


In [18]:
print( np.shape(X_train))
print(np.shape(Y_train)) 
print(np.shape(X_test)) 
print(np.shape(Y_test)) 

(50000, 32, 32, 3)
(50000, 100)
(10000, 32, 32, 3)
(10000, 100)


In [ ]:
from tensorflow import keras
model = keras.models.load_model(r"C:\Users\shaif\Downloads\Cifar100_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 100  # Update with the actual number of classes in your target data
x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model with early stopping
new_model.fit(X_train, Y_train,validation_split = 0.2, batch_size=32, epochs=50)

Epoch 1/50
1250/1250 [==============================] - 81s 61ms/step - loss: 4.3342 - accuracy: 0.1291 - val_loss: 22.0688 - val_accuracy: 0.1359
Epoch 2/50
1250/1250 [==============================] - 71s 57ms/step - loss: 4.2865 - accuracy: 0.1284 - val_loss: 18.6530 - val_accuracy: 0.1148
Epoch 3/50
1250/1250 [==============================] - 66s 53ms/step - loss: 3.7189 - accuracy: 0.1709 - val_loss: 6.1822 - val_accuracy: 0.2027
Epoch 4/50
1250/1250 [==============================] - 70s 56ms/step - loss: 3.5732 - accuracy: 0.2010 - val_loss: 936.2059 - val_accuracy: 0.0301
Epoch 5/50
1250/1250 [==============================] - 65s 52ms/step - loss: 3.6366 - accuracy: 0.1770 - val_loss: 5.7493 - val_accuracy: 0.1893
Epoch 6/50
1250/1250 [==============================] - 63s 51ms/step - loss: 3.1912 - accuracy: 0.2397 - val_loss: 4.8571 - val_accuracy: 0.2609
Epoch 7/50
1250/1250 [==============================] - 60s 48ms/step - loss: 2.9963 - accuracy: 0.2747 - val_loss: 3.29

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have trained a model and obtained predicted probabilities on x_test
y_pred_prob = new_model.predict(X_test)

# Convert predicted probabilities to predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert y_test to predicted labels format
y_test_labels = np.argmax(Y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score (micro-average)
f1_micro = f1_score(y_test_labels, y_pred, average='micro')
print("F1 Score (Micro):", f1_micro)